# Divide y Vencerás — Implementaciones en Python

## 1) Moda de un vector — DyV (mapas de frecuencia)

In [ ]:

from collections import Counter

def freq_map_divide_and_conquer(arr, base=64):
    n = len(arr)
    if n == 0: return Counter()
    if n <= base: return Counter(arr)
    mid = n // 2
    left = freq_map_divide_and_conquer(arr[:mid], base=base)
    right = freq_map_divide_and_conquer(arr[mid:], base=base)
    left.update(right)
    return left

def mode_divide_and_conquer(arr):
    freq = freq_map_divide_and_conquer(arr)
    if not freq: return (None, 0)
    value, count = max(freq.items(), key=lambda kv: kv[1])
    return value, count

data = [1,2,3,2,3,2,5,2,3,3,3,3,4,3,3,2,2,2,2]
print(mode_divide_and_conquer(data))


## 2) Multiplicación de enteros grandes — Karatsuba

In [ ]:

def karatsuba(x: int, y: int, base_case_digits: int = 4) -> int:
    if x < 0 or y < 0:
        raise ValueError("Use números no negativos para esta versión.")
    if x == 0 or y == 0: return 0
    nx = len(str(x)); ny = len(str(y))
    n = max(nx, ny)
    if n <= base_case_digits: return x * y
    m = n // 2
    high_x, low_x = divmod(x, 10**m)
    high_y, low_y = divmod(y, 10**m)
    z0 = karatsuba(low_x, low_y, base_case_digits)
    z2 = karatsuba(high_x, high_y, base_case_digits)
    z1 = karatsuba(high_x + low_x, high_y + low_y, base_case_digits) - z2 - z0
    return z2 * (10**(2*m)) + z1 * (10**m) + z0

a = int("12345678901234567890")
b = int("98765432109876543210")
print(karatsuba(a,b) == a*b)


## 3) Multiplicación de matrices — Strassen

In [ ]:

def next_power_of_two(n):
    return 1 if n == 0 else 2**(n-1).bit_length()

def add_matrix(A, B):
    n = len(A); m = len(A[0])
    return [[A[i][j] + B[i][j] for j in range(m)] for i in range(n)]

def sub_matrix(A, B):
    n = len(A); m = len(A[0])
    return [[A[i][j] - B[i][j] for j in range(m)] for i in range(n)]

def split_quadrants(M):
    n = len(M); mid = n//2
    A11 = [row[:mid] for row in M[:mid]]
    A12 = [row[mid:] for row in M[:mid]]
    A21 = [row[:mid] for row in M[mid:]]
    A22 = [row[mid:] for row in M[mid:]]
    return A11,A12,A21,A22

def join_quadrants(C11,C12,C21,C22):
    top = [c11 + c12 for c11, c12 in zip(C11, C12)]
    bot = [c21 + c22 for c21, c22 in zip(C21, C22)]
    return top + bot

def classic_multiply(A,B):
    n = len(A); p = len(B); m = len(B[0])
    C = [[0]*m for _ in range(n)]
    for i in range(n):
        for k in range(p):
            aik = A[i][k]
            if aik != 0:
                for j in range(m):
                    C[i][j] += aik * B[k][j]
    return C

def strassen(A, B, threshold=64):
    n, k = len(A), len(A[0])
    k2, m = len(B), len(B[0])
    assert k == k2, "Dimensiones incompatibles"
    size = max(n, k, m)
    s2 = next_power_of_two(size)
    A_pad = [[0]*s2 for _ in range(s2)]
    B_pad = [[0]*s2 for _ in range(s2)]
    for i in range(n):
        for j in range(k):
            A_pad[i][j] = A[i][j]
    for i in range(k2):
        for j in range(m):
            B_pad[i][j] = B[i][j]
    C_pad = _strassen_square(A_pad, B_pad, threshold)
    return [row[:m] for row in C_pad[:n]]

def _strassen_square(A, B, threshold):
    n = len(A)
    if n <= threshold:
        return classic_multiply(A,B)
    A11,A12,A21,A22 = split_quadrants(A)
    B11,B12,B21,B22 = split_quadrants(B)
    M1 = _strassen_square(add_matrix(A11,A22), add_matrix(B11,B22), threshold)
    M2 = _strassen_square(add_matrix(A21,A22), B11, threshold)
    M3 = _strassen_square(A11, sub_matrix(B12,B22), threshold)
    M4 = _strassen_square(A22, sub_matrix(B21,B11), threshold)
    M5 = _strassen_square(add_matrix(A11,A12), B22, threshold)
    M6 = _strassen_square(sub_matrix(A21,A11), add_matrix(B11,B12), threshold)
    M7 = _strassen_square(sub_matrix(A12,A22), add_matrix(B21,B22), threshold)
    C11 = add_matrix(sub_matrix(add_matrix(M1, M4), M5), M7)
    C12 = add_matrix(M3, M5)
    C21 = add_matrix(M2, M4)
    C22 = add_matrix(sub_matrix(add_matrix(M1, M3), M2), M6)
    return join_quadrants(C11,C12,C21,C22)

# Demo
A = [[1,2,3],[4,5,6]]
B = [[7,8],[9,10],[11,12]]
print(strassen(A,B) == classic_multiply(A,B))


## 4) Subsecuencia de suma máxima — DyV

In [ ]:

def max_crossing_sum(arr, mid):
    best = float("-inf"); s = 0
    for i in range(mid, -1, -1):
        s += arr[i]; best = max(best, s)
    left_best = best
    best = float("-inf"); s = 0
    for j in range(mid+1, len(arr)):
        s += arr[j]; best = max(best, s)
    right_best = best
    return left_best + right_best

def max_subarray_divide_and_conquer(arr):
    def rec(lo, hi):
        if lo == hi: return arr[lo]
        mid = (lo + hi)//2
        left = rec(lo, mid)
        right = rec(mid+1, hi)
        cross = max_crossing_sum(arr[lo:hi+1], mid-lo)
        return max(left, right, cross)
    if not arr: return 0
    return rec(0, len(arr)-1)

print(max_subarray_divide_and_conquer([13,-3,-25,20,-3,-16,-23,18,20,-7,12,-5,-22,15,-4,7]))
